## Calibração da Câmera usando OpenCV
- **Nome**: **Matheus Pacheco dos Santos**, **GRR**: **20192786**
- **Nome**: **João Gabriel Borges de Macedo**, **GRR**: **20190429**

### Introdução - Calibração da Câmera
Este notebook trata-se um exercício de Calibração da Câmera usando OpenCV. Tal exercício é um atividade de estudo de Visão Computacional.

A calibração da câmera é o processo de ajustar os parâmetros da câmera para que a imagem capturada corresponda com precisão à cena real. Esses parâmetros incluem a distância focal da lente, a posição e orientação da câmera em relação à cena e possíveis distorções ópticas.

Neste notebook introduzimos um pouco da Calibração da Câmera usando o **OpenCV** em python. Para tal calibração usamos 14 imagens de um tabuleiro de xadrez.

### Configuração Inicial
Usamos a função `cv.findChessboardCorners()` para encontrar os cantos. Definimos como padrão a grade 7x6

Depois de encontrar os cantos, podemos aumentar sua precisão usando `cv.cornerSubPix()`. Também podemos desenhar o padrão usando `cv.drawChessboardCorners()`. Todas essas etapas estão incluídas no código abaixo:

In [ ]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepara os pontos do objeto, como (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# arrays para armazenar pontos de objeto e pontos de imagem de todas as imagens
objpoints = [] # pontos 3d no espaço do mundo real
imgpoints = [] # pontos 2d no plano da imagem
imgs = glob.glob('./img/*.jpg')
for fname in imgs:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # encontra os cantos do tabuleiro de xadrez (ret indica se o padrão foi encontrado)
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)

    # Se encontrado, adicione pontos de objeto, pontos de imagem (depois de refiná-los)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        
        # Exibe os cantos
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        plt.imshow(img)
        plt.show()

### Calibração
Agora usamos a função `cv.calibrateCamera()` que retorna:
- **Matriz da câmera**; 
- **Coeficientes de distorção**; 
- **Vetores de rotação**;
- **Vetores de translação**.

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print('Matriz da Câmera')
print(mtx)
print('------------------------------------------')
print('Coeficientes de distorção')
print(dist)
print('------------------------------------------')
print('Vetores de rotação')
print(rvecs)
print('------------------------------------------')
print('Vetores de Translação')
print(tvecs)

Uma possível saída é:

- **Matriz da Câmera**
$$
Matriz = 
\begin{pmatrix}
841.04 & 0 & 299,82 \\
0 & 795.35 & 311.11 \\
0 & 0 & 1
\end{pmatrix}
$$
- **Coeficientes de Distorção**
$$
(k_1 \quad k_2 \quad p_1 \quad p_2 \quad k_3) = (-0.74 \quad 1.23 \quad -0.63 \quad 0.25 \quad -4.97)
$$
- **Vetores de Rotação**
$$
[[-0.68] \quad [0.38] \quad [-3.01]] \\
$$
- **Vetores de Translação**
$$
[[4.77] \quad [-1.27] \quad [22.63]] \\
$$


### Sem distorção
Agora, podemos pegar uma imagem e não distorcê-la. Primeiramente podemos refinar a matriz da câmera com base em um parâmetro de escala livre usando `cv.getOptimalNewCameraMatrix()`. Se o parâmetro de escala `alfa=0`, ele retorna uma imagem não distorcida com o mínimo de pixels indesejados. Portanto, pode até remover alguns pixels nos cantos da imagem. Se `alpha=1`, todos os pixels são retidos com algumas imagens pretas extras. Esta função também retorna um ROI de imagem que pode ser usado para cortar o resultado.

Então, pegamos uma nova imagem:

In [8]:
img = cv.imread('./img/left01.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

Agora basta calibramos a imagem com o método `cv.undistort()`

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# corta imagem
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('./img/left01-calibrated.jpg', dst)
plt.imshow(dst)
plt.show()